In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//span[@itemprop="name"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                price = html.xpath('//span[@data-label="Incl. Tax"]/span[@class="price"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                excl_tax = html.xpath('//span[@data-label="Excl. Tax"]/span[@class="price"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                                
                list_div = html.xpath('//div[@class="product-info-stock-sku"]/div')

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@alt="main product photo"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                details = '' if len(html.xpath('//div[@class="product attribute description"]/div/text()')) == 0 else html.xpath('//div[@class="product attribute description"]/div/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@id="product-attribute-specs-table"]/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                oe = '' if len(html.xpath('//div[@id="custom-1"]/text()')) == 0 else ';'.join(sorted(list(set([oe.strip() for oe in html.xpath('//div[@id="custom-1"]/text()')[0].strip().split('/')]))))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Price': price,
                                         'Excl_Tax': excl_tax,
                                         'Details': details,
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for div in list_div:
                    list_text = [text.strip() for text in list_div[0].xpath('./descendant-or-self::*/text()')]
                    while '' in list_text:
                        list_text.remove('')
                    df_temp.loc[0, list_text[0].strip().lower()] = list_text[1].strip()

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./th/text()')[0].strip()] = tr.xpath('./td/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：455

[ok] - https://en.euro-klima.pl/EN/kp010-kp010-kolo-pasowe-ekl30772-447280-1800.html
[尝试次数：2] - [剩余数量：435] - [当前时间：17:22:39]

[ok] - https://en.euro-klima.pl/EN/ca952-clutch-assembly.html
[尝试次数：3] - [剩余数量：434] - [当前时间：17:22:40]

[ok] - https://en.euro-klima.pl/EN/ca953-clutch-assembly.html
[尝试次数：3] - [剩余数量：433] - [当前时间：17:22:40]

[ok] - https://en.euro-klima.pl/EN/ca956-clutch-assembly.html
[尝试次数：8] - [剩余数量：432] - [当前时间：17:22:42]

[ok] - https://en.euro-klima.pl/EN/kp002-pulley-wheel-for-ca681.html
[尝试次数：8] - [剩余数量：431] - [当前时间：17:22:42]

[ok] - https://en.euro-klima.pl/EN/ca955-clutch-assembly-unicla-ux310-330u-2a-24v.html
[尝试次数：14] - [剩余数量：430] - [当前时间：17:22:43]

[ok] - https://en.euro-klima.pl/EN/ca945-clutch-assembly.html
[尝试次数：1] - [剩余数量：429] - [当前时间：17:22:44]

[ok] - https://en.euro-klima.pl/EN/ca946-clutch-assembly-unicla-ux-up200-10pk.html
[尝试次数：1] - [剩余数量：428] - [当前时间：17:22:44]

[ok] - https://en.euro-klima.pl/EN/kp007-kp007-kolo-pasowe-98800-92600-9911r.html
[尝试次数：9]

[ok] - https://en.euro-klima.pl/EN/ca874-clutch-assembly.html
[尝试次数：2] - [剩余数量：358] - [当前时间：17:23:15]

[ok] - https://en.euro-klima.pl/EN/ca877-clutch-assembly.html
[尝试次数：5] - [剩余数量：357] - [当前时间：17:23:15]

[ok] - https://en.euro-klima.pl/EN/ca872-clutch-assembly.html
[尝试次数：1] - [剩余数量：356] - [当前时间：17:23:15]

[ok] - https://en.euro-klima.pl/EN/ca873-clutch-assembly.html
[尝试次数：2] - [剩余数量：355] - [当前时间：17:23:15]

[ok] - https://en.euro-klima.pl/EN/ca875-clutch-assembly.html
[尝试次数：2] - [剩余数量：354] - [当前时间：17:23:16]

[ok] - https://en.euro-klima.pl/EN/ca871-clutch-assembly-1a-24v.html
[尝试次数：3] - [剩余数量：353] - [当前时间：17:23:16]

[ok] - https://en.euro-klima.pl/EN/ca880-clutch-assembly.html
[尝试次数：2] - [剩余数量：352] - [当前时间：17:23:17]

[ok] - https://en.euro-klima.pl/EN/ca867-clutch-assembly.html
[尝试次数：1] - [剩余数量：351] - [当前时间：17:23:18]

[ok] - https://en.euro-klima.pl/EN/ca866-clutch-assembly.html
[尝试次数：1] - [剩余数量：350] - [当前时间：17:23:20]

[ok] - https://en.euro-klima.pl/EN/ca865-clutch-assembly.html
[尝试次

[ok] - https://en.euro-klima.pl/EN/ca800-clutch-assembly.html
[尝试次数：1] - [剩余数量：279] - [当前时间：17:23:55]

[ok] - https://en.euro-klima.pl/EN/ca732-clutch-assembly-tm65-10pk-176mm-24v.html
[尝试次数：1] - [剩余数量：278] - [当前时间：17:23:56]

[ok] - https://en.euro-klima.pl/EN/ca947-clutch-assembly.html
[尝试次数：6] - [剩余数量：277] - [当前时间：17:23:57]

[ok] - https://en.euro-klima.pl/EN/ca727-clutch-assembly-1a-138mm-12v.html
[尝试次数：1] - [剩余数量：276] - [当前时间：17:23:57]

[ok] - https://en.euro-klima.pl/EN/ca72738-ca563-clutch-assembly.html
[尝试次数：1] - [剩余数量：275] - [当前时间：17:23:57]

[ok] - https://en.euro-klima.pl/EN/ca731-clutch-assembly-tm65-8pk-185mm-24v.html
[尝试次数：1] - [剩余数量：274] - [当前时间：17:23:57]

[ok] - https://en.euro-klima.pl/EN/ca728-clutch-assembly-tm32-10pk-176mm-24v.html
[尝试次数：1] - [剩余数量：273] - [当前时间：17:23:58]

[ok] - https://en.euro-klima.pl/EN/ca729-clutch-assembly-tm32-10pk-174mm-24v.html
[尝试次数：1] - [剩余数量：272] - [当前时间：17:23:58]

[ok] - https://en.euro-klima.pl/EN/kp009-kp009-kolo-pasowe-ekl39614-97701-1g

[ok] - https://en.euro-klima.pl/EN/ca660-clutch-assembly.html
[尝试次数：1] - [剩余数量：202] - [当前时间：17:24:38]

[ok] - https://en.euro-klima.pl/EN/ca661-clutch-assembly.html
[尝试次数：1] - [剩余数量：201] - [当前时间：17:24:38]

[ok] - https://en.euro-klima.pl/EN/ca657-clutch-assembly.html
[尝试次数：1] - [剩余数量：200] - [当前时间：17:24:38]

[ok] - https://en.euro-klima.pl/EN/ca653-clutch-assembly.html
[尝试次数：3] - [剩余数量：199] - [当前时间：17:24:40]

[ok] - https://en.euro-klima.pl/EN/ca655-clutch-assembly.html
[尝试次数：1] - [剩余数量：198] - [当前时间：17:24:40]

[ok] - https://en.euro-klima.pl/EN/ca656-clutch-assembly.html
[尝试次数：2] - [剩余数量：197] - [当前时间：17:24:40]

[ok] - https://en.euro-klima.pl/EN/ca652-clutch-assembly.html
[尝试次数：2] - [剩余数量：196] - [当前时间：17:24:41]

[ok] - https://en.euro-klima.pl/EN/ca649-clutch-assembly.html
[尝试次数：1] - [剩余数量：195] - [当前时间：17:24:41]

[ok] - https://en.euro-klima.pl/EN/ca650-clutch-assembly.html
[尝试次数：1] - [剩余数量：194] - [当前时间：17:24:42]

[ok] - https://en.euro-klima.pl/EN/ca676-clutch-assembly.html
[尝试次数：1] - 

[ok] - https://en.euro-klima.pl/EN/ca591-clutch-assembly.html
[尝试次数：8] - [剩余数量：122] - [当前时间：17:25:07]

[ok] - https://en.euro-klima.pl/EN/ca582-clutch-assembly.html
[尝试次数：1] - [剩余数量：121] - [当前时间：17:25:07]

[ok] - https://en.euro-klima.pl/EN/ca580-clutch-assembly-ca78576.html
[尝试次数：1] - [剩余数量：120] - [当前时间：17:25:07]

[ok] - https://en.euro-klima.pl/EN/ca58145-ca566-clutch-assembly-fs-fx.html
[尝试次数：3] - [剩余数量：119] - [当前时间：17:25:07]

[ok] - https://en.euro-klima.pl/EN/ca579-clutch-assembly-denso-ca78388.html
[尝试次数：1] - [剩余数量：118] - [当前时间：17:25:08]

[ok] - https://en.euro-klima.pl/EN/ca590-clutch-assembly.html
[尝试次数：1] - [剩余数量：117] - [当前时间：17:25:09]

[ok] - https://en.euro-klima.pl/EN/ca577-clutch-assembly-denso-ca78348.html
[尝试次数：1] - [剩余数量：116] - [当前时间：17:25:09]

[ok] - https://en.euro-klima.pl/EN/ca573-ca68451-zexel-dcw17-clutch-assembly.html
[尝试次数：2] - [剩余数量：115] - [当前时间：17:25:09]

[ok] - https://en.euro-klima.pl/EN/ca575-clutch-assembly-sanden-sd7v16-ca68530.html
[尝试次数：3] - [剩余数量：114] 

[ok] - https://en.euro-klima.pl/EN/ca310-clutch-assembly-denso.html
[尝试次数：1] - [剩余数量：44] - [当前时间：17:25:33]

[ok] - https://en.euro-klima.pl/EN/ca502-zexel-12v-8pk-clutch-assembly.html
[尝试次数：4] - [剩余数量：43] - [当前时间：17:25:33]

[ok] - https://en.euro-klima.pl/EN/ca501-zexel-12v-tm-2a-clutch-assembly.html
[尝试次数：6] - [剩余数量：42] - [当前时间：17:25:34]

[ok] - https://en.euro-klima.pl/EN/ca308-clutch-assembly-denso-110mm-4pk.html
[尝试次数：1] - [剩余数量：41] - [当前时间：17:25:34]

[ok] - https://en.euro-klima.pl/EN/ca503-zexel-24v-tm-8pk-clutch-assembly.html
[尝试次数：2] - [剩余数量：40] - [当前时间：17:25:34]

[ok] - https://en.euro-klima.pl/EN/ca500-zexel-24v-tm-2a-clutch-assembly.html
[尝试次数：6] - [剩余数量：39] - [当前时间：17:25:35]

[ok] - https://en.euro-klima.pl/EN/ca307-clutch-assembly-denso-115mm-6pk.html
[尝试次数：4] - [剩余数量：38] - [当前时间：17:25:35]

[ok] - https://en.euro-klima.pl/EN/ca303-clutch-assembly.html
[尝试次数：1] - [剩余数量：37] - [当前时间：17:25:35]

[ok] - https://en.euro-klima.pl/EN/ca301-98835-clutch-assembly.html
[尝试次数：2] - [剩余数